# Analys av könsfördelningen bland styrelseledamöter i svenska aktiebolag
av [Anna Norberg](https://twitter.com/annanorberg)

Hur ser könsfördelningen ut bland styrelseledamöter i aktiebolag i landets kommuner? Och hur såg den ut för tio år sedan? Nyhetsbyrån Siren har gjort en analys av Bolagsverkets öppna data om företrädare i svenska företag, som finns dokumenterad nedan.

Resultatet från analysen har rapporterats i en artikel som skickades ut till Nyhetsbyrån Sirens kunder den 9 maj 2018.

## Så här gjorde vi
### Importera Pythons analysverktyg

In [1]:
import pandas as pd

### Läs in data från Bolagsverkets hemsida

In [2]:
heads = pd.read_csv("http://www.bolagsverket.se/statistik/foretradare_historik.csv", engine='python', delimiter='\;')

In [3]:
heads.head()

,Ar,Foretagsform,Lan,Kommun,PrivatPublikt,Arbetstagarrepresentant,Utlandsbosatt,Bosatt_EES,Kon,Alder_intervall,...,Antal_REVS,Antal_REVSL,Antal_REVST,Antal_REVT,Antal_SU,Antal_SVD,Antal_VD,Antal_VLE,Antal_VOF,Antal_VVD
0,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Kvinna,50 �r och �ldre,...,10,0,0,0,221,0,6,0,0,2
1,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Kvinna,Yngre �n 50 �r,...,18,1,0,0,180,0,8,0,0,0
2,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Man,50 �r och �ldre,...,51,11,0,0,197,0,100,0,0,3
3,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Man,Yngre �n 50 �r,...,214,0,0,0,128,0,87,0,0,5
4,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Org,Ok�nd,...,0,0,0,0,0,0,0,0,0,0


### Förbered datan för analys
Filtrera ut endast aktiebolag, aktuellt år och företag med fysiska företrädare. Summera antalet styrelseledamöter per kön och kommun. Beräkna totalt antal styrelseledamöter, samt andelen kvinnor och män.

In [4]:
 aktiebolag = heads[heads.Foretagsform == "AB"]

In [5]:
ab_arton = aktiebolag[aktiebolag.Ar == 2018]
ab_arton_fys = ab_arton[ab_arton.JurFys == "F"]
ab_ledamot = ab_arton_fys.groupby(["Kommun", "Kon"]).Antal_LE.sum().reset_index()
kommuner_2018 = ab_ledamot.pivot(index="Kommun", columns="Kon")
pd.set_option('precision', 0)
kommuner_2018.drop(kommuner_2018.columns[[2]], axis=1, inplace=True)
kommuner_2018.loc['Summa']= kommuner_2018.sum()
kommuner_2018['Totalt'] = kommuner_2018.sum(axis=1)
kommuner_2018.columns = ['Kvinna_18', 'Man_18', 'Totalt_18']
kommuner_2018["Andel_kvinna_18"] = (kommuner_2018.Kvinna_18 / kommuner_2018.Totalt_18)*100
kommuner_2018["Andel_man_18"] = (kommuner_2018.Man_18 / kommuner_2018.Totalt_18)*100
kommuner_2018.head()

,Kvinna_18,Man_18,Totalt_18,Andel_kvinna_18,Andel_man_18
Kommun,,,,,
Ale kommun,240,1053,1293,19,81
Alings�s kommun,510,2443,2953,17,83
Alvesta kommun,173,876,1049,16,84
Aneby kommun,41,309,350,12,88
Arboga kommun,126,606,732,17,83


Gör samma sak med 2008.

In [6]:
ab_atta = aktiebolag[aktiebolag.Ar == 2008]
ab_atta_fys = ab_atta[ab_atta.JurFys == "F"]
ab_ledamot = ab_atta_fys.groupby(["Kommun", "Kon"]).Antal_LE.sum().reset_index()
kommuner_2008 = ab_ledamot.pivot(index="Kommun", columns="Kon")
pd.set_option('precision', 0)
kommuner_2008.drop(kommuner_2008.columns[[2]], axis=1, inplace=True)
kommuner_2008.loc['Summa']= kommuner_2008.sum()
kommuner_2008['Totalt'] = kommuner_2008.sum(axis=1)
kommuner_2008.columns = ['Kvinna_08', 'Man_08', 'Totalt_08']
kommuner_2008["Andel_kvinna_08"] = (kommuner_2008.Kvinna_08 / kommuner_2008.Totalt_08)*100
kommuner_2008["Andel_man_08"] = (kommuner_2008.Man_08 / kommuner_2008.Totalt_08)*100
kommuner_2008.head()

,Kvinna_08,Man_08,Totalt_08,Andel_kvinna_08,Andel_man_08
Kommun,,,,,
Ale kommun,114,665,779,15,85
Alings�s kommun,276,1506,1782,15,85
Alvesta kommun,111,725,836,13,87
Aneby kommun,35,228,263,13,87
Arboga kommun,72,436,508,14,86


### Lägg ihop de två dataseten

In [7]:
forandring = pd.merge(kommuner_2008, kommuner_2018, left_index=True, right_index=True)

### Beräkna förändringen

In [8]:
forandring["Forandring_kvinna"] = forandring.Andel_kvinna_18 - forandring.Andel_kvinna_08

In [9]:
forandring.head()

,Kvinna_08,Man_08,Totalt_08,Andel_kvinna_08,Andel_man_08,Kvinna_18,Man_18,Totalt_18,Andel_kvinna_18,Andel_man_18,Forandring_kvinna
Kommun,,,,,,,,,,,
Ale kommun,114,665,779,15,85,240,1053,1293,19,81,4
Alings�s kommun,276,1506,1782,15,85,510,2443,2953,17,83,2
Alvesta kommun,111,725,836,13,87,173,876,1049,16,84,3
Aneby kommun,35,228,263,13,87,41,309,350,12,88,-2
Arboga kommun,72,436,508,14,86,126,606,732,17,83,3


### Rangordna kommunerna
Här är andelen kvinnor lägst 2018:

In [10]:
 forandring.sort_values("Andel_kvinna_18").head()

,Kvinna_08,Man_08,Totalt_08,Andel_kvinna_08,Andel_man_08,Kvinna_18,Man_18,Totalt_18,Andel_kvinna_18,Andel_man_18,Forandring_kvinna
Kommun,,,,,,,,,,,
Aneby kommun,35,228,263,13,87,41,309,350,12,88,-2e+00
Vetlanda kommun,174,1382,1556,11,89,209,1509,1718,12,88,1e+00
S�vsj� kommun,70,579,649,11,89,98,698,796,12,88,2e+00
Bjurholm kommun,15,120,135,11,89,18,121,139,13,87,2e+00
Dals-Ed kommun,29,175,204,14,86,44,291,335,13,87,-1e+00


In [11]:
 forandring.sort_values("Andel_kvinna_18", ascending=False).head()

,Kvinna_08,Man_08,Totalt_08,Andel_kvinna_08,Andel_man_08,Kvinna_18,Man_18,Totalt_18,Andel_kvinna_18,Andel_man_18,Forandring_kvinna
Kommun,,,,,,,,,,,
Liding� kommun,726,2436,3162,23,77,1460,3712,5172,28,72,5
Danderyd kommun,1093,3194,4287,25,75,1766,4624,6390,28,72,2
Jokkmokk kommun,62,261,323,19,81,118,311,429,28,72,8
Dorotea kommun,33,104,137,24,76,48,127,175,27,73,3
M�rbyl�nga kommun,122,535,657,19,81,243,673,916,27,73,8


Här har andelen kvinnor sjunkit mest mellan 2008 och 2018:

In [12]:
 forandring.sort_values("Forandring_kvinna").head()

,Kvinna_08,Man_08,Totalt_08,Andel_kvinna_08,Andel_man_08,Kvinna_18,Man_18,Totalt_18,Andel_kvinna_18,Andel_man_18,Forandring_kvinna
Kommun,,,,,,,,,,,
Sorsele kommun,43,150,193,22,78,41,174,215,19,81,-3
Ludvika kommun,153,724,877,17,83,200,1130,1330,15,85,-2
Vaxholm kommun,155,500,655,24,76,337,1205,1542,22,78,-2
Skinnskatteberg kommun,23,108,131,18,82,27,144,171,16,84,-2
Berg kommun,81,323,404,20,80,110,491,601,18,82,-2


Här har andelen kvinnor ökat mest mellan 2008 och 2018:

In [13]:
 forandring.sort_values("Forandring_kvinna", ascending=False).head()

,Kvinna_08,Man_08,Totalt_08,Andel_kvinna_08,Andel_man_08,Kvinna_18,Man_18,Totalt_18,Andel_kvinna_18,Andel_man_18,Forandring_kvinna
Kommun,,,,,,,,,,,
Mal� kommun,33,191,224,15,85,48,145,193,25,75,10
Gr�storp kommun,17,147,164,10,90,63,263,326,19,81,9
Gnesta kommun,49,258,307,16,84,146,443,589,25,75,9
Storfors kommun,10,112,122,8,92,32,158,190,17,83,9
Sj�bo kommun,96,608,704,14,86,249,874,1123,22,78,9


### Exportera det nya datasetet till csv

In [14]:
 forandring.to_csv("styrelse-forandring.csv", decimal=',', sep=',', float_format= '%.0f')